##Code to extract list of edges from wordnet data dump.
##Currently considering it as an unlabeled graph, but can easily add labeling back in.

In [1]:
import os,pdb
import numpy as np
import pandas as pd
%pdb

Automatic pdb calling has been turned ON


In [9]:
OUT_DIR = '/Users/vmisra/Dropbox/IBM/code/kg_embedding/embedding_theano/wordnet/'
DATA_DIR = '/Users/vmisra/Downloads/wordnet'

noun_path = os.path.join(DATA_DIR,'data.noun')
verb_path = os.path.join(DATA_DIR,'data.verb')
out_path = os.path.join(OUT_DIR,'noun_relations.pkl')

print out_path

/Users/vmisra/Dropbox/IBM/code/kg_embedding/embedding_theano/wordnet/noun_relations.pkl


In [10]:
MIN_ENTITY_COUNT = 1
EARLY_STOPPAGE = 100000000

ERROR! Session/line number was not unique in database. History logging moved to new session 145


In [11]:
graph = []
idxtoname = {}

In [12]:
for line in open(noun_path,'r'):
    if len(graph)>EARLY_STOPPAGE:
        break
    if line.startswith(' '): #header detected
        continue

    line = line.partition('|')[0] #strip the example
    splits = line.split(' ')

    #first, build the dictionary from entityA's idx to list of words_sensenum
    entityA = splits[0]
    nwords = int(splits[3],16)
    names = ''
    for nword in range(nwords):
        idx_word = 4+2*nword
        names+=splits[idx_word]+'_'+splits[idx_word+1]+';'
    idxtoname[entityA] = names

    idx_entitiesB = 4+2*nwords
    n_entitiesB = int(splits[idx_entitiesB])

    for which_entityB in range(n_entitiesB):
        idx_entityB = idx_entitiesB + 2 + which_entityB*4
        entityB = splits[idx_entityB]
        if splits[idx_entityB+1]=='n': #only paying attention to noun-noun edges for now
            graph.append([entityA,entityB])

In [13]:
reldata = pd.DataFrame(graph,columns=['entityA','entityB'])
entities = pd.DataFrame(reldata.values.flatten(),columns = ['entity'])
entities_count = pd.DataFrame(entities.groupby('entity').size(),columns=['count']).reset_index()
entities_count = entities_count[entities_count['count']>=MIN_ENTITY_COUNT]
entities_count['idx'] = range(len(entities_count))
entity_to_index = entities_count.drop('count',1).set_index('entity').to_dict()['idx']
#idxToIDX = dict(zip(idxtoname.keys(),range(len(idxtoname))))
#print "idxToIDX: ", idxToIDX

In [14]:
def Atest(row):
    key = row['entityA']
    try:
        return entity_to_index[key]
    except KeyError,e:
        #print 'error'
        return -1
def Btest(row):
    key = row['entityB']
    try:
        return entity_to_index[key]
    except KeyError,e:
        #print 'error'
        return -1

In [15]:
reldata['Aindex'] = reldata.apply(func = Atest,axis=1)
reldata['Bindex'] = reldata.apply(func = Btest,axis=1)
reldata = reldata.drop(labels=['entityA','entityB'],axis = 1, inplace=False)
reldata = reldata[(~(reldata['Aindex']==-1))&(~(reldata['Bindex']==-1))]
reldata

,Aindex,Bindex
0,0,1
1,0,2
2,0,24647
3,1,0
4,1,3
5,1,4
6,1,16
7,1,24
8,1,42
9,1,78104


In [16]:
data = reldata[['Aindex','Bindex']].as_matrix()
import cPickle as pickle
pickle.dump((data,idxtoname,entity_to_index),open(out_path,'w'))
print data

[[    0     1]
 [    0     2]
 [    0 24647]
 ..., 
 [82114 81623]
 [82114  6383]
 [82114  3882]]


In [17]:
len(data)

231535